In [1]:
from pypylon import pylon
import platform

img = pylon.PylonImage()
tlf = pylon.TlFactory.GetInstance()

cam = pylon.InstantCamera(tlf.CreateFirstDevice())
cam.Open()

In [2]:
num_img_to_save = 20
cam.Gain = 20
cam.ExposureTime = 20000
cam.StartGrabbing()

for i in range(num_img_to_save):
    with cam.RetrieveResult(2000) as result:

        # Calling AttachGrabResultBuffer creates another reference to the
        # grab result buffer. This prevents the buffer's reuse for grabbing.
        img.AttachGrabResultBuffer(result)

        # if platform.system() == 'Windows':
        #     # The JPEG format that is used here supports adjusting the image
        #     # quality (100 -> best quality, 0 -> poor quality).
        #     ipo = pylon.ImagePersistenceOptions()
        #     quality = 90 - i * 10
        #     ipo.SetQuality(quality)

        #     filename = "saved_pypylon_img_%d.jpeg" % quality
        #     img.Save(pylon.ImageFileFormat_Jpeg, filename, ipo)
        # else:
        filename = "saved_pypylon_img_%d.png" % i
        img.Save(pylon.ImageFileFormat_Png, filename)

        # In order to make it possible to reuse the grab result for grabbing
        # again, we have to release the image (effectively emptying the
        # image object).
        img.Release()

cam.StopGrabbing()
cam.Close()

InvalidArgumentException: Cannot save image. The passed image is invalid. : InvalidArgumentException thrown (file 'imagepersistance.cpp', line 258)

In [1]:
# ===============================================================================
#    This sample illustrates how to grab and process images using the CInstantCamera class.
#    The images are grabbed and processed asynchronously, i.e.,
#    while the application is processing a buffer, the acquisition of the next buffer is done
#    in parallel.
#
#    The CInstantCamera class uses a pool of buffers to retrieve image data
#    from the camera device. Once a buffer is filled and ready,
#    the buffer can be retrieved from the camera object for processing. The buffer
#    and additional image data are collected in a grab result. The grab result is
#    held by a smart pointer after retrieval. The buffer is automatically reused
#    when explicitly released or when the smart pointer object is destroyed.
# ===============================================================================
from pypylon import pylon

img = pylon.PylonImage()
# Number of images to be grabbed.
countOfImagesToGrab = 10
# Create an instant camera object with the camera device found first.
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()

camera.GainAuto.TrySetValue( pylon.AutoGainOff )
camera.Gain.SetValue( camera.Gain.GetMin() )
camera.Gain = 5
camera.ExposureTime = 90000

# Print the model name of the camera.
print("Using device ", camera.GetDeviceInfo().GetModelName())

# demonstrate some feature access
new_width = camera.Width.GetValue() - camera.Width.GetInc()
if new_width >= camera.Width.GetMin():
    camera.Width.SetValue(new_width)

# The parameter MaxNumBuffer can be used to control the count of buffers
# allocated for grabbing. The default value of this parameter is 10.
camera.MaxNumBuffer = 10

# Start the grabbing of c_countOfImagesToGrab images.
# The camera device is parameterized with a default configuration which
# sets up free-running continuous acquisition.
camera.StartGrabbingMax(countOfImagesToGrab)

i = 0
# Camera.StopGrabbing() is called automatically by the RetrieveResult() method
# when c_countOfImagesToGrab images have been retrieved.

while camera.IsGrabbing():
    # Wait for an image and then retrieve it. A timeout of 5000 ms is used.
    grabResult = camera.RetrieveResult(10000000, pylon.TimeoutHandling_ThrowException)

    img.AttachGrabResultBuffer(grabResult)
    filename = "saved_pypylon_G5_E9_img_%d.png" % i
    img.Save(pylon.ImageFileFormat_Png, filename)
    
    i += 1

    # In order to make it possible to reuse the grab result for grabbing
    # again, we have to release the image (effectively emptying the
    # image object).
    img.Release()

    # Image grabbed successfully?
    if grabResult.GrabSucceeded():
        # Access the image data.
        print("SizeX: ", grabResult.Width)
        print("SizeY: ", grabResult.Height)
        imgShow = grabResult.Array
        print("Gray value of first pixel: ", imgShow[0, 0])
    else:
        print("Error: ", grabResult.ErrorCode, grabResult.ErrorDescription)
    grabResult.Release()

camera.Close()

RuntimeException: No device is available or no device contains the provided device info properties. : RuntimeException thrown (file 'tlfactory.cpp', line 694)

In [5]:
camera.Close()